## **Python notebook: Índices Espectrais - Bioma da Caatinga** 🛰🔥
---







## **Instalando e importando bibliotecas**
---









In [ ]:
#!pip install geospatial
#Esta linha de código é responsável por realizar a instalação de diversos pacotes comummente utilizados em geoprocessamento e produção de gráficos
#Já este link apresenta as orientações de como utilizar pacotes gráficos: https://python-graph-gallery.com/

In [ ]:
!pip install geemap

In [ ]:
!pip install eemont
!pip install wxee
!pip install seaborn proplot matplotlib==3.4.0
!pip install geopandas

In [ ]:
import ee
import eemont
import geemap
import json
import geopandas as gpd
import pandas as pd
import numpy as np
import geemap.colormaps as cm
import matplotlib.pyplot as plt
ee.Authenticate()
ee.Initialize()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Autorizando acesso ao Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **`Área de interesse`**
---



In [ ]:
roi=gpd.read_file('/content/drive/MyDrive/TCC_Shapefiles/TI_ATIKUM.shp')
roi=roi.to_json()
roi=json.loads(roi)
ATIKUM=ee.FeatureCollection(roi)

# **`Análise MODIS`**
---


In [ ]:
MOD09GA = ee.ImageCollection('MODIS/006/MOD09GA') \
          .filterDate('2020-10-01','2021-10-30') \
          .preprocess()

In [ ]:
MOD09GA = MOD09GA.spectralIndices('NDVI')

In [ ]:
ndvi_viz = {'min':0, 'max':1, 'palette': cm.palettes.RdYlGn, 'bands':'NDVI'}

In [ ]:
Map = geemap.Map(location=[-21.31, -46.7617], zoom=4)
Map.addLayer(PIMENTEIRA, {'opacity': 0.75}, 'UCE - Parque Estadual da Mata Pimenteira')
Map.addLayer(MOD09GA.median().clip(PIMENTEIRA), ndvi_viz, 'NDVI')
Map.add_colorbar(ndvi_viz, label='NDVI', orientation='vertical')
Map

# **`Análise LANDSAT 8`**
---

## **`Análise LANDSAT 8 - Xarray`**


In [ ]:
import proplot as plot
import wxee
import xarray as xr

In [ ]:
roi=gpd.read_file('/content/drive/MyDrive/TCC_Shapefiles/TI_ATIKUM.shp')
roi=roi.to_json()
roi=json.loads(roi)
ATIKUM=ee.FeatureCollection(roi)

In [ ]:
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
             .filterBounds(ATIKUM) \
             .scale() \
             .preprocess()\
             .maskClouds() \
             .spectralIndices()

In [ ]:
dset = landsat8.filterDate('2021-10-15', '2021-11-15').first().select('NDVI').clip(ATIKUM).wx.to_xarray(region=ATIKUM.geometry(), scale=30)

In [ ]:
dset

In [ ]:
fig, ax = plot.subplots(figsize=(10,7), tight=True)

# mapa de contornos preenchidos
map1 = ax.contourf(dset['x'], dset['y'], dset['NDVI'].squeeze(),
                   cmap='RdYlGn', levels=plot.arange(-1, 1, 0.05)) 

# barra de cores
fig.colorbar(map1, loc='r', label='NDVI', labelsize=13)

# opções de formatação
ax.format(xlabel='Longitude', ylabel='Latitude', title='Território Indígena Atikum - NDVI',
          xformatter='deglon', yformatter='deglat',
          xtickminor=True, ytickminor=True, 
          small='10px', large='25px')

# salvar a figura
#fig.save('guaxupe_ndvi.jpeg', dpi=300)

plot.show()

## **`Análise LANDSAT 8 - Geemap`**

In [ ]:
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
             .filterBounds(ATIKUM) \
             .scale() \
             .preprocess() \
             .maskClouds() \

In [ ]:
#Pré Fogo
prefire_start = '2020-12-20';   
prefire_end = '2021-01-18';
#Pós Fogo
postfire_start = '2021-02-01';
postfire_end = '2021-04-28';

In [ ]:
landsat8prefire= landsat8.filterDate(prefire_start,prefire_end)

In [ ]:
landsat8postfire= landsat8.filterDate(postfire_start,postfire_end)

In [ ]:
pre_mos = landsat8prefire.mosaic().clip(ATIKUM)

In [ ]:
post_mos = landsat8postfire.mosaic().clip(ATIKUM)

In [ ]:
PRE_NBR=pre_mos.normalizedDifference(['B5', 'B7']);

In [ ]:
POST_NBR=post_mos.normalizedDifference(['B5', 'B7']);

In [ ]:
dNBR= (PRE_NBR.subtract(POST_NBR)).multiply(1000);
dnbr_viz = {'min':0, 'max':1, 'palette': cm.palettes.RdYlGn_r}

In [ ]:
Map = geemap.Map(location=[-8.31, -38.7617], zoom=12)
Map.addLayer(dNBR,dnbr_viz)
Map

Map(center=[-8.31, -38.7617], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

## **`Análise LANDSAT 8 - Estatísticas`**

In [ ]:
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
              .filterBounds(ATIKUM).scale().maskClouds().spectralIndices('EVI').spectralIndices('NDVI')

In [ ]:
lista_indices = landsat8.first().bandNames().getInfo()

In [ ]:
lista_indices

In [ ]:
indices_series = landsat8.getTimeSeriesByRegion(geometry = ATIKUM,
                                                bands = lista_indices,
                                                reducer = ee.Reducer.mean(),
                                                scale = 30)

In [ ]:
indices_series = geemap.ee_to_pandas(indices_series)

In [ ]:
indices_series.head()

,reducer,date,B1,B2,B3,B4,B5,B6,B7,B10,B11,sr_aerosol,pixel_qa,radsat_qa,EVI,NDVI
0,mean,2013-04-02T12:45:50,0.024400,0.033974,0.066708,0.084074,0.244055,0.262827,0.170794,303.850114,299.824956,93.314493,322.006124,0.0,0.273714,0.509350
1,mean,2013-04-21T12:43:36,0.020358,0.029784,0.059596,0.063046,0.227217,0.201299,0.124778,296.306500,292.850763,178.270144,322.479486,0.0,0.298821,0.585394
2,mean,2013-05-23T12:43:52,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.0,-9999.000000,-9999.000000
3,mean,2013-06-08T12:43:51,0.028229,0.036324,0.067158,0.071252,0.273508,0.244151,0.147311,296.162090,292.880312,174.827006,322.438338,0.0,0.356766,0.631248
4,mean,2013-06-24T12:43:45,0.025981,0.034648,0.061513,0.077114,0.228407,0.251144,0.168083,301.236172,299.238089,92.986126,322.006062,0.0,0.269910,0.520732


In [ ]:
indices_series[indices_series == -9999.] = np.nan
indices_series = indices_series.dropna()

In [ ]:
indices_series.head()

,reducer,date,B1,B2,B3,B4,B5,B6,B7,B10,B11,sr_aerosol,pixel_qa,radsat_qa,EVI,NDVI
0,mean,2013-04-02T12:45:50,0.024400,0.033974,0.066708,0.084074,0.244055,0.262827,0.170794,303.850114,299.824956,93.314493,322.006124,0.0,0.273714,0.509350
1,mean,2013-04-21T12:43:36,0.020358,0.029784,0.059596,0.063046,0.227217,0.201299,0.124778,296.306500,292.850763,178.270144,322.479486,0.0,0.298821,0.585394
3,mean,2013-06-08T12:43:51,0.028229,0.036324,0.067158,0.071252,0.273508,0.244151,0.147311,296.162090,292.880312,174.827006,322.438338,0.0,0.356766,0.631248
4,mean,2013-06-24T12:43:45,0.025981,0.034648,0.061513,0.077114,0.228407,0.251144,0.168083,301.236172,299.238089,92.986126,322.006062,0.0,0.269910,0.520732
5,mean,2013-07-10T12:43:49,0.046096,0.063135,0.117056,0.144529,0.304836,0.346227,0.230760,292.019609,288.947922,221.474659,325.184113,0.0,0.238315,0.370854


In [ ]:
indices_series['date'] = pd.to_datetime(indices_series['date'],
                                        infer_datetime_format = 'Y%-m%-d%',)

In [ ]:
indices_series = indices_series.set_index(indices_series['date'])

In [ ]:
indices_series.head()

,reducer,date,B1,B2,B3,B4,B5,B6,B7,B10,B11,sr_aerosol,pixel_qa,radsat_qa,EVI,NDVI
date,,,,,,,,,,,,,,,,
2013-04-02 12:45:50,mean,2013-04-02 12:45:50,0.024400,0.033974,0.066708,0.084074,0.244055,0.262827,0.170794,303.850114,299.824956,93.314493,322.006124,0.0,0.273714,0.509350
2013-04-21 12:43:36,mean,2013-04-21 12:43:36,0.020358,0.029784,0.059596,0.063046,0.227217,0.201299,0.124778,296.306500,292.850763,178.270144,322.479486,0.0,0.298821,0.585394
2013-06-08 12:43:51,mean,2013-06-08 12:43:51,0.028229,0.036324,0.067158,0.071252,0.273508,0.244151,0.147311,296.162090,292.880312,174.827006,322.438338,0.0,0.356766,0.631248
2013-06-24 12:43:45,mean,2013-06-24 12:43:45,0.025981,0.034648,0.061513,0.077114,0.228407,0.251144,0.168083,301.236172,299.238089,92.986126,322.006062,0.0,0.269910,0.520732
2013-07-10 12:43:49,mean,2013-07-10 12:43:49,0.046096,0.063135,0.117056,0.144529,0.304836,0.346227,0.230760,292.019609,288.947922,221.474659,325.184113,0.0,0.238315,0.370854


In [ ]:
indices_mensais = indices_series.groupby(pd.Grouper(freq='1M')).mean()

In [ ]:
indices_mensais

In [ ]:
fig, ax = plot.subplots(figsize=(15, 10), tight=True)

ax.plot(indices_mensais.index, indices_mensais['NDVI'],
        color='green', marker='o', label='NDVI')

ax.format(xlabel='Anos', ylabel='Valor',
          title='NDVI - TI Atikum - Séries Mensais de 2013 a 2022',
          xrotation=0, xtickminor=False, ytickminor=False,
          small='15px', large='20px', grid=False)

ax.legend(loc='bottom', frameon=False)

plot.show()

In [ ]:
indices_mensais_interp = indices_mensais.interpolate(method='polynomial', order=2)

In [ ]:
indices_mensais_interp

,B1,B2,B3,B4,B5,B6,B7,B10,B11,sr_aerosol,pixel_qa,radsat_qa,EVI,NDVI
date,,,,,,,,,,,,,,
2013-04-30,0.022379,0.031879,0.063152,0.073560,0.235636,0.232063,0.147786,300.078307,296.337860,135.792319,322.242805,0.000000e+00,0.286267,0.547372
2013-05-31,0.021146,0.028403,0.052505,0.059534,0.237901,0.221270,0.140511,301.158052,298.445014,98.509963,321.546445,-8.439557e-24,0.324738,0.613173
2013-06-30,0.027105,0.035486,0.064335,0.074183,0.250958,0.247647,0.157697,298.699131,296.059200,133.906566,322.222200,0.000000e+00,0.313338,0.575990
2013-07-31,0.037844,0.050307,0.092101,0.109422,0.267659,0.300776,0.195532,294.860879,291.574476,210.836150,323.714463,0.000000e+00,0.260197,0.456526
2013-08-31,0.038849,0.054662,0.095573,0.117793,0.249218,0.319467,0.225496,300.821325,297.402874,167.907085,322.885805,0.000000e+00,0.215755,0.382851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,0.029623,0.042586,0.072279,0.087170,0.223368,0.258077,0.169920,296.269793,293.722890,163.807395,365.800179,0.000000e+00,0.239787,0.450782
2021-09-30,0.032635,0.044407,0.073262,0.099220,0.220139,0.297969,0.196454,302.268919,298.830436,155.062324,322.398879,0.000000e+00,0.204461,0.392875
2021-10-31,0.034849,0.047514,0.074583,0.103413,0.213809,0.312250,0.212913,305.948363,302.011786,127.894839,322.001445,0.000000e+00,0.187668,0.360031


In [ ]:
fig, ax = plot.subplots(figsize=(10, 5), tight=True)

ax.plot(indices_mensais_interp.index, indices_mensais_interp['NDVI'],
        color='green', marker='x', label='NDVI')

ax.format(xlabel='Anos', ylabel='Valor',
          title='NDVI - TI Atikum - Séries Mensais de 2013 a 2021',
          xrotation=0, xtickminor=False, ytickminor=False,
          small='15px', large='20px', grid=False)

ax.legend(loc='bottom', frameon=False)

plot.show()

### ***`NDVI - TI Atikum - Heatmap`***

In [ ]:
import seaborn as sns

In [ ]:
ndvi = np.reshape(indices_mensais_interp['NDVI']['2014-01-01':'2021-12-31'].values, (8, 12), order='C')

In [ ]:
# deixando o proplot de lado e voltando para o bom e velho matplotlib
fig, ax = plt.subplots(figsize=(10, 5))

# criando heatmap com seaborn
sns.heatmap(ndvi, vmin=0, vmax=1, cmap='RdYlGn',
            xticklabels=['JAN', 'FEV', 'MAR', 'ABR', 'MAI', 'JUN','JUL', 'AGO', 'SET', 'OUT', 'NOV', 'DEZ'],
            yticklabels=plot.arange(2014, 2021, 1), 
            linewidth=0.5, linecolor='black',
            cbar_kws={'label': 'NDVI'}, annot=True,)

plt.tight_layout()
plt.show()

## **`Análise LANDSAT 8 - Impacto pós queimada`**

In [ ]:
import proplot as plot
import wxee
import xarray as xr

In [ ]:
roi=gpd.read_file('/content/drive/MyDrive/TCC_Shapefiles/TI_ATIKUM.shp')
roi=roi.to_json()
roi=json.loads(roi)
ATIKUM=ee.FeatureCollection(roi)

In [ ]:
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
             .filterBounds(ATIKUM) \
             .scale() \
             .preprocess()\
             .maskClouds() \

In [ ]:
#Houve um incêndio no dia 2021/10/15

landsat8_set_2020 = landsat8.filterDate('2021-08-01', '2021-10-01').median()
landsat8_out_2020 = landsat8.filterDate('2021-10-14', '2021-12-14').median()

In [ ]:
#Bandas
preNBR =landsat8_set_2020.normalizedDifference(['B5', 'B7']).clip(ATIKUM)
postNBR = landsat8_out_2020.normalizedDifference(['B5', 'B7']).clip(ATIKUM)

In [ ]:
DELTANBR=preNBR.subtract(postNBR)

In [ ]:
print(DELTANBR.getInfo())

{'type': 'Image', 'bands': [{'id': 'nd', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [ ]:
Map = geemap.Map(location=[-8.28, -38.7617], zoom=12)
viz =  {'min':-0.5, 'max':0.5, 'palette': cm.palettes.RdYlGn_r}
Map.add_colorbar(viz, label='dNBR', orientation='horizontal')
Map.addLayer(DELTANBR,viz,'dNBR')
Map

Map(center=[-8.28, -38.7617], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [ ]:
#Sugestão do André! Analisar!
latlon = ee.Image.pixelLonLat().addBands(DELTANBR)

In [ ]:
#Exportar imagem para o Drive em formato tiff

region= ee.Geometry.Rectangle(-39.00,-8.20,-38.7,-8.4)#Importante fazer isso para cada nova área analisada
task = ee.batch.Export.image.toDrive(**{
    'image':DELTANBR,
    'description':'Image export 2',
    'folder':'TESTE',
    'scale':30,
    'region': region.getInfo()['coordinates']
})
task.start()

### Com esse comando consigo exportar a imagem e gerar todos meus plots pelo QGGIS/ArcMap!!!

***Cálculo de área - Impacto pós queimada***

In [ ]:
NBRALTO=DELTANBR.gt(-2)

In [ ]:
NBRALTO_MASK=NBRALTO.clip(ATIKUM).selfMask()

In [ ]:
Map = geemap.Map(location=[-8.29, -38.7617], zoom=12)
viz =  {'min':-0.5, 'max':0.5, 'palette': cm.palettes.RdYlGn_r}
Map.addLayer(NBRALTO_MASK,viz)
Map.add_colorbar(viz, label='dNBR', orientation='horizontal')
Map

Map(center=[-8.29, -38.7617], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [ ]:
allpix=NBRALTO_MASK.reduce('sum').toInt()

In [ ]:
pixstats = allpix.reduceRegion(**{
  'reducer': ee.Reducer.count(),               
  'geometry': ATIKUM,
  'scale': 30 #Resolução espacial do Satélite
  });

In [ ]:
allpixels = ee.Number(pixstats.get('sum'))

In [ ]:
NPixels=allpixels.getInfo()

In [ ]:
print("O número de pixels nas condições indicadas é: ",NPixels)

O número de pixels nas condições indicadas é:  184326


In [ ]:
print("A área, em hectares, é:",NPixels*30*30/1e4)

A área, em hectares, é: 16589.34


## **`Composição de banda - SENTINEL 2A - 12,8A,4`**

In [ ]:
roi=gpd.read_file('/content/drive/MyDrive/TCC_Shapefiles/TI_ATIKUM.shp')
roi=roi.to_json()
roi=json.loads(roi)
ATIKUM=ee.FeatureCollection(roi)

In [ ]:
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR") \
             .filterBounds(ATIKUM) \
             .preprocess()\
             .maskClouds() \

In [ ]:
sentinel2_out_2020 = sentinel2.filterDate('2021-10-01', '2021-10-30').median().clip(ATIKUM)

In [ ]:
FALSACOR = sentinel2_out_2020.select('B12','B8A','B4')

In [ ]:
FALSACOR_VIZ = {'min':0.019, 'max': 0.19, 
           'bands':['B12','B8A','B4']}

In [ ]:
Map = geemap.Map(location=[-8.31, -38.7617], zoom=12)
Map.addLayer(FALSACOR,FALSACOR_VIZ)
Map

In [ ]:
#Exportar imagem para o Drive em formato tiff

region= ee.Geometry.Rectangle(-39.00,-8.20,-38.7,-8.4)#Importante fazer isso para cada nova área analisada
task = ee.batch.Export.image.toDrive(**{
    'image':FALSACOR,
    'description':'FALSACOR',
    'folder':'TESTE',
    'scale':10,
    'region': region.getInfo()['coordinates']
})
task.start()

### Com esse comando consigo exportar a imagem e gerar todos meus plots pelo QGGIS/ArcMap!!!

In [ ]:
#Exportar imagem para o Drive em formato tiff

region= ee.Geometry.Rectangle(-39.00,-8.20,-38.7,-8.4)#Importante fazer isso para cada nova área analisada
task = ee.batch.Export.image.toDrive(**{
    'image':sentinel2_out_2020,
    'description':'sentinel2_out_2020_falsacor',
    'folder':'TESTE',
    'scale':10,
    'region': region.getInfo()['coordinates']
})
task.start()

### Com esse comando consigo exportar a imagem e gerar todos meus plots pelo QGGIS/ArcMap!!!